# Video Embedding Basics

<div style="background-color: #d9f2f7ff; border:1px solid #a0a0a060; padding: 1rem; color: #151515">
✏️ <strong>Note:</strong> The examples below can be run in any order. Input and output will be automatically saved to the "output" folder.
</div>

Video embeddings can be generated asynchronously using the Amazon Bedrock Runtime **InvokeModel** function or asynchronously using the **StartAsyncInvoke** function.

### Synchronous Embedding - Inline Video Data

This first example includes the video data directly within the request using the `source.bytes` request parameter.


In [ ]:
# Restore variables from setup notebook
%store -r s3_bucket
print(f"Using S3 bucket: {s3_bucket}")
%store -r region_name
print(f"Using region: {region_name}")

In [ ]:
import json

from utils.utils import load_file_as_base64
import nova_embeddings

request_body = {
    "taskType": "SINGLE_EMBEDDING",
    "singleEmbeddingParams": {
        "embeddingPurpose": "GENERIC_INDEX",
        "embeddingDimension": 256,
        "video": {
            "format": "mp4",
            "embeddingMode": "AUDIO_VIDEO_COMBINED",
            "source": {
                "bytes": load_file_as_base64("sample_assets/videos/the-sea.mp4")
            },
        },
    },
}

body, metadata = nova_embeddings.generate_embedding_sync(request_body)

print("Request ID:", metadata.get("RequestId"))
print(json.dumps(body, indent=2))

### Synchronous Emedding - S3 Video File

This next example creates an embedding for a video file stored in S3.

In [ ]:
import json

import nova_embeddings

# Update this constant to point to a video file in S3.
S3_VIDEO_FILE = f"s3://{s3_bucket}/sample_assets/videos/the-sea.mp4"

request_body = {
    "taskType": "SINGLE_EMBEDDING",
    "singleEmbeddingParams": {
        "embeddingPurpose": "GENERIC_INDEX",
        "embeddingDimension": 3072,
        "video": {
            "format": "mp4",
            "embeddingMode": "AUDIO_VIDEO_COMBINED",
            "source": {
                "s3Location": {
                    "uri": S3_VIDEO_FILE,
                }
            },
        },
    },
}

body, metadata = nova_embeddings.generate_embedding_sync(request_body)

print("Request ID:", metadata.get("RequestId"))
print(json.dumps(body, indent=2))

### Asynchronous Embedding - Inline Video Data

Use the **StartAsyncInvoke** function to generate an embedding asynchronously. (See the `generate_embedding_async()` function definition in the "nova_embeddings.py" file for full details.) The results will be stored to an S3 bucket you specify. When generating an async embedding for video files, you can specify the `segmentationConfig.durationSeconds` value to dictate how longer videos will be broken into segments. The default is 5 seconds. 

Calls to StartAsyncInvoke will return an `invocationArn` that can be used to check the status of the async job using the **GetAsyncInvoke** function.

Edit the `S3_DESTINATION_BUCKET` value below to point to your own S3 bucket. Then run the cell.

In [ ]:
from utils.utils import load_file_as_base64
import nova_embeddings

# Update this constant to reference your own S3 bucket.
S3_DESTINATION_BUCKET = s3_bucket

request_body = {
    "taskType": "SEGMENTED_EMBEDDING",
    "segmentedEmbeddingParams": {
        "embeddingPurpose": "GENERIC_INDEX",
        "embeddingDimension": 256,
        "video": {
            "format": "mp4",
            "embeddingMode": "AUDIO_VIDEO_COMBINED",
            "source": {
                "bytes": load_file_as_base64("sample_assets/videos/the-sea.mp4")
            },
            "segmentationConfig": {"durationSeconds": 5},
        },
    },
}

invocation_arn, metadata = nova_embeddings.generate_embedding_async(
    request_body, S3_DESTINATION_BUCKET
)

print("Request ID:", metadata.get("RequestId"))
print("Invocation ARN:", invocation_arn)

You can use the **GetAsyncInvoke** function to check the status of your asynchronous embedding job.

In [ ]:
from utils.utils import pretty_format
import boto3

# Uncomment the line below if you'd like to use a hard-coded invocation_arn.
# invocation_arn = "<your-invocation-arn>"

if "invocation_arn" not in locals():
    raise SystemExit(
        "invocation_arn variable not found. Please run the previous cell first or hard code an invocation_arn in this cell."
    )

# Create the Bedrock Runtime client.
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name=region_name,
)

try:
    # Get the async job status.
    response = bedrock_runtime.get_async_invoke(
        invocationArn=invocation_arn,
    )

except Exception as e:
    # You would typically add your own exception handling here.
    print(e)

print(pretty_format(response))

The convenience function below will download the output for recent async jobs to the "output" directory. It will optionally monitor any in-progress jobs, downloading their output when they complete.

In [ ]:
from utils.async_job_utils import download_recent_async_jobs

# Download and monitor async jobs from the past N hours.
download_recent_async_jobs(output_dir="output", hours=4, wait_for_running_jobs=True)